In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

from gc import collect as collect_garbage
from psutil import virtual_memory
from os import scandir

In [2]:
%matplotlib widget

In [3]:
collect_garbage()
virtual_memory()

svmem(total=12655771648, available=6495129600, percent=48.7, used=6160642048, free=6495129600)

In [4]:
coldict = {
    "raw_waves_data_1a.csv": ["257"], "raw_waves_data_1b.csv": ["257", "258"], "raw_waves_data_1c.csv": ["257", "258"], "raw_waves_data_1d.csv": ["257", "258", "317"], 
    "raw_waves_data_1e.csv": ["258"],

    "raw_waves_data_2a.csv": ["257", "258"], "raw_waves_data_2b.csv": ["258"], "raw_waves_data_2c.csv": ["257"], "raw_waves_data_2d.csv": ["257", "258"], 
    "raw_waves_data_2e.csv": ["257", "258"],

    "raw_waves_data_3a.csv": ["258"], "raw_waves_data_3b.csv": ["258"], "raw_waves_data_3c.csv": ["258"], "raw_waves_data_3d.csv": ["258"], 
    "raw_waves_data_3e.csv": ["257", "258", "317"],

    "raw_waves_data_4a.csv": ["257", "258"], "raw_waves_data_4b.csv": ["257", "258"], "raw_waves_data_4c.csv": ["257"], "raw_waves_data_4d.csv": ["257", "258"], 
    "raw_waves_data_4e.csv": ["257", "258"],

    "raw_waves_data_5a.csv": ["258"], "raw_waves_data_5b.csv": ["258"], "raw_waves_data_5c.csv": ["258"], "raw_waves_data_5d.csv": ["258", "317"],
    "raw_waves_data_5e.csv": ["258"],

    "raw_waves_data_6a.csv": ["257", "258"], "raw_waves_data_6b.csv": ["258"], "raw_waves_data_6c.csv": ["258"], "raw_waves_data_6d.csv": ["258"], "raw_waves_data_6e.csv": ["258"],
    
    "raw_waves_data_7a.csv": ["257", "258"], "raw_waves_data_7b.csv": ["258"], "raw_waves_data_7c.csv": ["258"], "raw_waves_data_7d.csv": ["257", "258", "317"], 
    "raw_waves_data_7e.csv": ["258"]
}

In [5]:
ecg_folder = "../../Deidentified-Raw-Waveforms/"
rpeak_folder = "completed_rpeaks/"

In [6]:
# We got the file for infant 1 2 3 as a test run, so for now we can leave that one out
for i in range(4,8):
    good_peaks = []
    all_filt2 = []
    shift = 0
    for part in ["a", "b", "c", "d", "e"]:
        print("Now working on " + str(i) + part + "; " + str( virtual_memory()[2] ) + "% memory use")

        # Load in the raw data
        file = "raw_waves_data_" + str(i) + part + ".csv"
        cols = coldict[file]
        df = pd.read_csv(ecg_folder + file, usecols=cols)
        print("Data loaded in" + "; " + str( virtual_memory()[2] ) + "% memory use")

        # Construct the completed ECG signal
        signal = pd.Series(df[cols[0]])
        j=1
        while True:
            try:
                signal = signal.combine_first(df[cols[j]])
                j+=1
            except IndexError:
                break
        signal = signal.fillna(0)
        signal = pd.to_numeric(signal)
        print("Signals combined and filled in" + "; " + str( virtual_memory()[2] ) + "% memory use")

        # Delete df
        del df
        collect_garbage()
        print("Dataframe deleted"+ "; " + str( virtual_memory()[2] ) + "% memory use")

        # Load in the rpeak found before as well
        rpeaks = pd.read_csv(rpeak_folder + "test_rpeaks_" + str(i) + part + ".csv")
        rpeaks.columns = ["sample", "time"]
        print("R-peaks loaded in" + "; " + str( virtual_memory()[2] ) + "% memory use")

        # Calculate filters
        # filt1 {True if rpeak is not in spike, False otherwise}
        filt1 = pd.Series([True for i in range(len(rpeaks["time"]))])
        for j in range(len(filt1)):
            ecg_value = signal[rpeaks.loc[j, "sample"]]
            if ecg_value < -18 or ecg_value > 50:
                filt1[j] = False
        
        # Can delete signal now
        del signal
        collect_garbage()

        # Apply filt1 to rpeaks and append this to goodpeaks
        good_peaks.append( rpeaks.loc[filt1] )

        # Now, we're using the name filt2, but this is really a list of indices where "post-spike" peaks land, just don't want to replace the variable name everywhere
        filt2 = []
        for j in range(1, len(rpeaks)):
            if filt1[j-1] == False and filt1[j] == True:
                filt2.append(j + shift)
        all_filt2 += filt2
        shift += len(rpeaks)

        print("Filters calculated" + "; " + str( virtual_memory()[2] ) + "% memory use")

        # We can delete filt1 now
        del filt1
        del rpeaks
        collect_garbage()
        print("Filter 1 and rpeaks deleted" + "; " + str( virtual_memory()[2] ) + "% memory use")
    
    # At this point, we need to append all of the signals to one dataframe
    # I'll do this the sloppy way for now, if it's too slow, then I'll try and do it without a quadratic copy
    all_peaks = good_peaks[0]
    #big_filt2 = all_filt2[0]
    for j in range(1,5):
        all_peaks = all_peaks.append(good_peaks[j])
        #big_filt2.append(all_filt2[j])
    all_peaks.reset_index(inplace=True)
    print("All R-peaks concatenated and reindexed" + "; " + str( virtual_memory()[2] ) + "% memory use")

    final_peaks = pd.DataFrame( all_peaks["time"] )
    final_peaks.reset_index(inplace=True)
    final_peaks["rrivl"] = all_peaks.diff()["time"]
    
    all_filt2 = sorted(list(set(range(len(final_peaks["time"]))) - set(all_filt2)))
    final_peaks.loc[all_filt2, ["time", "rrivl"]].to_csv("troughless-rpeaks/filtered_rpeaks_" + str(i) + ".csv", index=False)

    del all_peaks
    del all_filt2
    del final_peaks
    collect_garbage()
    print("Output file written" + "; " + str( virtual_memory()[2] ) + "% memory use")
    print()
    

        






Now working on 4a; 48.7% memory use
Data loaded in; 56.2% memory use
Signals combined and filled in; 60.5% memory use
Dataframe deleted; 52.3% memory use
R-peaks loaded in; 52.4% memory use
Filters calculated; 49.8% memory use
Filter 1 and rpeaks deleted; 49.4% memory use
Now working on 4b; 49.4% memory use
Data loaded in; 64.6% memory use
Signals combined and filled in; 68.8% memory use
Dataframe deleted; 60.5% memory use
R-peaks loaded in; 60.4% memory use
Filters calculated; 55.6% memory use
Filter 1 and rpeaks deleted; 55.1% memory use
Now working on 4c; 55.1% memory use
Data loaded in; 61.8% memory use
Signals combined and filled in; 65.9% memory use
Dataframe deleted; 61.9% memory use
R-peaks loaded in; 59.8% memory use
Filters calculated; 54.8% memory use
Filter 1 and rpeaks deleted; 54.3% memory use
Now working on 4d; 54.3% memory use
Data loaded in; 62.7% memory use
Signals combined and filled in; 66.9% memory use
Dataframe deleted; 58.7% memory use
R-peaks loaded in; 59.0% me

In [19]:
final_peaks.index.value_counts()

0         5
449721    5
449703    5
449704    5
449705    5
         ..
704228    1
704227    1
704226    1
704225    1
710007    1
Length: 710008, dtype: int64

TypeError: unsupported operand type(s) for -: 'list' and 'list'